In [2]:
#imports
import os
import pandas as pd

os.chdir('./solar_data')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor
import numpy as np
import itertools



from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.ets import AutoETS

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

seed_value = 123

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable: './solar_data'

In [2]:
energy_production = pd.read_csv('Solar_Energy_Production.csv')
energy_production_sites = pd.read_csv('Solar_Photovoltaic_Sites.csv')


In [ ]:
energy_production_sites.head(5)

In [ ]:
energy_production

### Null values 

In [ ]:
nulls = pd.DataFrame(energy_production.isna().mean() * 100, columns=['percentage_of_nulls'])
nulls

In [ ]:
nulls_sites = pd.DataFrame(energy_production_sites.isna().mean() * 100, columns=['percentage_of_nulls'])
nulls_sites

### check if each id in your DataFrame corresponds to only one site_name

In [ ]:
energy_production.groupby('id')['name'].agg(lambda x: ', '.join(set(x)))

In [ ]:
energy_production_sites.groupby('id')['name'].agg(lambda x: ', '.join(set(x)))

In [ ]:
array2 = set(energy_production['id'].unique())
array1 = set(energy_production_sites['id'].unique())
same_elements = set(array1) - set(array2)
same_elements

### Merge datasets on id

In [10]:
energy_production.columns
energy_production = energy_production[['name', 'id', 'address', 'date', 'kWh']]

In [ ]:
energy_production_sites.columns
energy_production_sites = energy_production_sites[['id', 'latitude', 'longitude']]
energy_production_sites

In [ ]:
merged_df = pd.merge(energy_production, energy_production_sites,on='id', how='left')
merged_df

#### convert date column to datetime 

In [18]:
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df.rename(columns={'kWh': 'GHI'}, inplace=True)
merged_df = merged_df.sort_values(by=['name', 'date'])


### visualisation 

In [ ]:
m1 = merged_df[merged_df['id'] == 164440 ]
m1

In [ ]:
m2 = m1.sort_values(by='date')
m2

In [ ]:
fig = px.line(
    merged_df, 
    x='date', 
    y='GHI', 
    color='name', 
    facet_col='name',  
    facet_col_wrap=2,  
    title="Solar Generation Time Series for All Sites"
)

fig.update_layout(
    xaxis_title='Datetime',
    yaxis_title='Solar Generation (kWh)',
    template='plotly_dark',  
    height=900,  # Adjust height to accommodate multiple plots
    width=1200
)

# Show the plot
fig.show()

In [31]:
import openmeteo_requests

import requests_cache
from retry_requests import retry
from datetime import datetime
import requests

In [32]:
def api(start_date, end_date, latitude, longitude):
    """api call function.

    Keyword arguments:
    time -- desired date in "%Y-%m-%d format
    latitude -- gps coordinate
    longitude -- gps coordinate
    """

    date_string_1 = start_date + "T" + "00:00"
    date_string_2 = end_date + "T" + "00:00"

    date_object1 = datetime.strptime(date_string_1, "%Y-%m-%dT%H:%M")
    date_object2 = datetime.strptime(date_string_2, "%Y-%m-%dT%H:%M")

    date_only1 = date_object1.date()
    date_only2 = date_object2.date()
    date_string_1 = str(date_only1)
    date_string_2 = str(date_only2)
    # import dataset from API
    x = [(latitude, longitude)]

    li = []
    for i in x:
        params = {
            "latitude": i[0],
            "longitude": i[1],
            "start_date": date_string_1,
            "end_date": date_string_2,
            "timezone": "auto",
            "temperature_unit": "fahrenheit",  #  units
            "windspeed_unit": "mph",
            "precipitation_unit	": "inch",
            "hourly": {
                "precipitation",
                "snowfall",
                "temperature_2m",
                "relativehumidity_2m",
                "surface_pressure",
                "windspeed_10m",
                "winddirection_10m",
                "windgusts_10m",
                "cloudcover",
            },
            "daily": {"sunrise", "sunset"},
        }

        response = requests.get(
            "https://archive-api.open-meteo.com/v1/era5", params=params
        )
        res = response.json()


    return  res

In [33]:
start_time = str('2023-12-04')
# last time stamp
end_time = str('2024-01-04')
lat = 46.999873
lon = 6.498147
res = api(start_time, end_time, lat, lon)

In [ ]:
res

In [ ]:
merged_df

In [40]:
def api(start_date, end_date, latitude, longitude):
    """api call function.

    Keyword arguments:
    time -- desired date in "%Y-%m-%d format
    latitude -- gps coordinate
    longitude -- gps coordinate
    """

    date_string_1 = start_date + "T" + "00:00"
    date_string_2 = end_date + "T" + "00:00"

    date_object1 = datetime.strptime(date_string_1, "%Y-%m-%dT%H:%M")
    date_object2 = datetime.strptime(date_string_2, "%Y-%m-%dT%H:%M")

    date_only1 = date_object1.date()
    date_only2 = date_object2.date()
    date_string_1 = str(date_only1)
    date_string_2 = str(date_only2)
    # import dataset from API
    x = [(latitude, longitude)]

    li = []
    for i in x:
        params = {
            "latitude": i[0],
            "longitude": i[1],
            "start_date": date_string_1,
            "end_date": date_string_2,
            "timezone": "auto",
            "temperature_unit": "fahrenheit",  #  units
            "windspeed_unit": "mph",
            "precipitation_unit	": "inch",
            "hourly": {
                "precipitation",
                "snowfall",
                "temperature_2m",
                "relativehumidity_2m",
                "surface_pressure",
                "windspeed_10m",
                "winddirection_10m",
                "windgusts_10m",
                "cloudcover",
            },
            "daily": {"sunrise", "sunset"},
        }

        response = requests.get(
            "https://archive-api.open-meteo.com/v1/era5", params=params
        )
        res = response.json()

        df3 = pd.DataFrame.from_dict(res["daily"], orient="index").T
        df3 = df3.loc[df3.index.repeat(24)].reset_index(drop=True)
        df2 = pd.DataFrame.from_dict(res["hourly"], orient="index").T
        df2 = df2.assign(
            elevation=res["elevation"],
            latitude=res["latitude"],
            longitude=res["longitude"],
            timezone=res["timezone_abbreviation"],
        )
        df2["sunrise"] = df3["sunrise"]
        df2["sunset"] = df3["sunset"]
        li.append(df2)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame, res

In [ ]:
grouped_conc = merged_df.groupby("id")
frames = []
for id, group in grouped_conc:

    
    start_time = str(group["date"].iloc[0].date())
    print(start_time)
    end_time = str(group["date"].iloc[len(group) - 1].date())
    print(end_time)
    lat = group["latitude"].iloc[0]
    print(lat)
    lon = group["longitude"].iloc[0]
    print(lon)
    frame ,res = api(start_time, end_time, lat, lon)

    frames.append(frame)

In [ ]:
frame

In [43]:
def predict(data):
    """Solar radiation prediction function and weather data extraction.

    Keyword arguments:

    data -- merged hourly data frame from inventory and SBAP using merging fonction
    """

    data["snowfall"] = 0
    data["precipitation"] = 0
    data["cloudcover"] = 0

    data["surface_pressure"] = 0
    data["winddirection_10m"] = 0
    data["windgusts_10m"] = 0
    data["windspeed_10m"] = 0
    data["relativehumidity_2m"] = 0
    data["temperature_2m"] = 0
    data["elevation"] = 0
    data["Solar_radiation"] = 0

    devices = []

    grouped_conc = data.groupby("id")

    for id, group in grouped_conc:

        start_time = str(group["date"].iloc[0].date())
        # last time stamp
        end_time = str(group["date"].iloc[len(group) - 1].date())
        lat = group["latitude"].iloc[0]
        lon = group["longitude"].iloc[0]
        frame, res = api(start_time, end_time, lat, lon)
        for i in range(len(group)):

            times = str(group["date"].iloc[i].date())

            hour2 = group["date"].iloc[i].strftime("%H:00")

            snowfall = frame.query("time=='{}'".format(times + "T" + hour2))[
                "snowfall"
            ].values[0]
            precipitation = frame.query("time=='{}'".format(times + "T" + hour2))[
                "precipitation"
            ].values[0]
            cloudcover = frame.query("time=='{}'".format(times + "T" + hour2))[
                "cloudcover"
            ].values[0]

            surface_pressure = frame.query("time=='{}'".format(times + "T" + hour2))[
                "surface_pressure"
            ].values[0]
            winddirection_10m = frame.query("time=='{}'".format(times + "T" + hour2))[
                "winddirection_10m"
            ].values[0]
            windgusts_10m = frame.query("time=='{}'".format(times + "T" + hour2))[
                "windgusts_10m"
            ].values[0]
            windspeed_10m = frame.query("time=='{}'".format(times + "T" + hour2))[
                "windspeed_10m"
            ].values[0]
            relativehumidity_2m = frame.query("time=='{}'".format(times + "T" + hour2))[
                "relativehumidity_2m"
            ].values[0]
            temperature_2m = frame.query("time=='{}'".format(times + "T" + hour2))[
                "temperature_2m"
            ].values[0]
            elevation = frame.query("time=='{}'".format(times + "T" + hour2))[
                "elevation"
            ].values[0]

            group["snowfall"].iloc[i] = snowfall
            group["cloudcover"].iloc[i] = cloudcover

            group["surface_pressure"].iloc[i] = surface_pressure
            group["winddirection_10m"].iloc[i] = winddirection_10m
            group["windspeed_10m"].iloc[i] = windspeed_10m

            group["relativehumidity_2m"].iloc[i] = relativehumidity_2m
            group["temperature_2m"].iloc[i] = temperature_2m
            group["elevation"].iloc[i] = elevation

           

        devices.append(group)

    conc = pd.concat(devices, ignore_index=True)

    return conc

In [ ]:
concatenated = predict(merged_df)